![](./resources/System_v1_inference_header.png)

This notebook contains a short demo on how to use the WorldCereal system to generate a cropland extent and crop type map,
using default models trained by the WorldCereal consortium.

# Content

- [Before you start](#before-you-start)
- [1. Define region of interest](#1.-Define-a-region-of-interest)
- [2. Generate default Worldcereal products](#2.-Generate-default-WorldCereal-products)
- [3. Explore available reference data](#3.-Explore-available-reference-data)

# Before you start

In order to run this notebook, you need to create an account on the Copernicus Data Space Ecosystem,
by completing the form [HERE](https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=eRKGqDvoYI0).


In [1]:
# First we import the necessary modules to run this notebook

import pandas as pd
import requests
from pathlib import Path
from shapely.geometry import shape, Polygon

import leafmap

from openeo_gfmap import TemporalContext, BoundingBoxExtent
from openeo_gfmap.backend import Backend, BackendContext

import sys
sys.path.append('/home/jovyan/worldcereal-classification/src')

from worldcereal.utils.map import (get_ui_map, _latlon_to_utm, 
                                   visualize_product)
from worldcereal.job import generate_map
from worldcereal.seasons import get_processing_dates_for_extent

RDM_API = 'https://ewoc-rdm-api.iiasa.ac.at'
LOCALTILESERVER_PORT = 3334

# 1. Define a region of interest

When running the code snippet below, an interactive map will be visualized.
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.
When finished, execute the second cell to store the coordinates of your region of interest. 

In [ ]:
m, dc = get_ui_map()
m

In [ ]:
# retrieve bounding box from drawn rectangle
obj = dc.last_draw
if obj.get('geometry') is not None:
    poly = Polygon(shape(obj.get('geometry')))
    bbox = poly.bounds
else:
    raise ValueError('Please first draw a rectangle '
                     'on the map before proceeding.')
print(f'Your area of interest: {bbox}')

# We convert our bounding box to local UTM projection for further processing
bbox_utm, epsg = _latlon_to_utm(bbox)
spatial_extent = BoundingBoxExtent(*bbox_utm, epsg)

# 2. Generate default WorldCereal products

In this section we will generate two WorldCereal mapping products for our region of interest using a series of pre-trained models made available by the consortium:
1) A temporary crops extent product, defined as a binary mask indicating the presence of temporary crops.
2) A binary seasonal crop type product, indicating for a specific growing season which pixels previously identified as temporary crops contain our crop of interest (in this case maize).

We start with the temporary crops product. This product is generated for a full year. The exact timing of the processing period is defined by the WorldCereal crop calendars.

In [ ]:
#---------------------------------------------

# DEFINE INPUTS FOR GENERATING THE MAP
year = 2021
season = 'tc-annual'
output_path = '/vitodata/worldcereal/test/temporal_crops_v2.tif'
# output_path = '/home/joyvan/test/temporal_crops.nc'

#---------------------------------------------

# Get the processing period for the temporary crops product based on the WorldCereal crop calendars:
start_date, end_date = get_processing_dates_for_extent(spatial_extent, year, season)

# Defining period of interest for temporary crops product
temporal_extent = TemporalContext(start_date, end_date)

# We specify the cloud backend where computations need to be done
backend_context = BackendContext(Backend.FED)

# We call the function to generate the map on the CDSE backend using OpenEO
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
generate_map(spatial_extent, temporal_extent,
             backend_context, output_path)

The resulting map will be written to disk (specified by output_path), but you can also visualize the map directly through the OpenEO web editor on the Copernicus Data Space Ecosystem, by logging in [HERE](https://openeo.dataspace.copernicus.eu/) with your CDSE account details.

### IMPORTANT INFORMATION TO ENSURE DISPLAY OF RESULTS WORKS PROPERLY

if working on binder, set localtileserver client prefix
import os
os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = "proxy/{{port}}"

if working on terrascope virtual machine, ensure that you forward the port of the localtileserver
1) in the add_raster function, add the following argument: port=LOCALTILESERVER_PORT
2) ensure that whichever number you defined as the LOCALTILESERVER_PORT, this port is forwarded to your local machine
e.g. Port 7777, Forwarded address: localhost:7778


In [2]:
infile = '/vitodata/worldcereal/test/openEO_2020-01-01Z.tif'
m = visualize_product(infile, product='temporary-crops', port=LOCALTILESERVER_PORT)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

# 3. Explore available reference data

In-situ reference data used for training and validating WorldCereal classification models is hosted in the WorldCereal Reference Data Module (RDM). This module can be accessed through a dedicated user interface (https://ewoc-rdm-ui.iiasa.ac.at/map), but also through an API, which will be demonstrated here.

In this section we will explore the available reference datasets for our region of interest. In case you would like to test multiple regions of interest, execute the first two cells in [this section](#1.-Define-a-region-of-interest) of the notebook.

In [ ]:
# Check full list of available collections
headers = {}
collectionResponse = requests.get(f'{RDM_API}/collections', headers=headers)
collections = collectionResponse.json()
col_ids = [x['collectionId'] for x in collections['items']]
print(f'Available collections: {col_ids}')

# Now we check which collections intersect with our AOI
bbox_str = f'Bbox={bbox[0]}&Bbox={bbox[1]}&Bbox={bbox[2]}&Bbox={bbox[3]}'
colSearchUrl = f'{RDM_API}/collections/search?{bbox_str}'
colSearchResponse = requests.get(colSearchUrl, headers=headers)
test = colSearchResponse.json()
print('The following collections intersect with your AOI:')
for i, col in enumerate(test):
    print()
    print(f'Collection {i+1}: {col["collectionId"]}')
